In [4]:
###getting the reports local url's:###
##extracting folders id from shared memory

from google.colab import auth
from googleapiclient.discovery import build
import pandas as pd

# Authenticate
auth.authenticate_user()
drive_service = build('drive', 'v3')

folder_id = "1e0eA-AIsz_BSwVHOppJMXECX42hBfG4J"

def list_all_files_in_folder_recursive(parent_id, parent_path=""):
    """Recursively list all files in a folder and subfolders"""
    all_files = []

    query = f"'{parent_id}' in parents and trashed=false"
    page_token = None

    while True:
        response = drive_service.files().list(
            q=query,
            spaces='drive',
            fields='nextPageToken, files(id, name, mimeType)',
            pageToken=page_token
        ).execute()

        for item in response.get('files', []):
            item_path = f"{parent_path}/{item['name']}" if parent_path else item['name']

            if item['mimeType'] == 'application/vnd.google-apps.folder':
                # Recurse into subfolder
                all_files.extend(list_all_files_in_folder_recursive(item['id'], item_path))
            else:
                all_files.append({
                    "file_name": item['name'],
                    "file_path": item_path,
                    "file_id": item['id'],
                    "file_url": f"https://drive.google.com/file/d/{item['id']}/view?usp=sharing"
                })

        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break

    return all_files

# Run the recursive function
files_list = list_all_files_in_folder_recursive(folder_id)

# Convert to DataFrame
df_files = pd.DataFrame(files_list)
##df_files.head()



##getting local id of the file
!pip install python-docx
import os
from googleapiclient.http import MediaIoBaseDownload
import io

def get_chapter_file(chapter_num: int, year: str, download_dir="/content") -> str:
    """
    Download a chapter file for a given year from Google Drive and return its local path.

    Args:
        chapter_num (int): The chapter number (e.g., 1, 2, ..., 14).
        year (str): The year folder name.
        download_dir (str): Local folder to save the file. Defaults to /content in Colab.

    Returns:
        str: Local path to the downloaded file, or None if not found.
    """
    # Normalize chapter filename (01, 02, ...)
    chapter_str = f"{chapter_num:02}"

    # Find matching file in df_files
    match = df_files[
        (df_files["file_name"].str.contains(chapter_str)) &
        (df_files["file_path"].str.contains(f"{year}/"))
    ]

    if match.empty:
        print(f"❌ No file found for chapter {chapter_str} in year {year}")
        return None

    file_id = match.iloc[0]["file_id"]
    file_name = match.iloc[0]["file_name"]
    local_path = os.path.join(download_dir, year, file_name)

    # Ensure year directory exists locally
    os.makedirs(os.path.dirname(local_path), exist_ok=True)

    # Download the file from Google Drive
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.FileIO(local_path, "wb")
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()
        if status:
            print(f"⬇️ Download {int(status.progress() * 100)}%.")

    print(f"✅ Saved to {local_path}")
    return local_path

In [6]:
import os
import io
from googleapiclient.http import MediaIoBaseDownload

def download_all_chapters(download_dir="/content/reports", years=range(2001, 2025), chapters=range(1, 16)):
    """
    Downloads all chapters for all years to local environment.

    Returns:
        dict: {year: {chapter_number: local_path}}
    """
    all_paths = {}

    for year in years:
        year_str = str(year)
        all_paths[year_str] = {}

        for chapter in chapters:
            chapter_str = f"{chapter:02}"

            # Find matching file in df_files
            match = df_files[
                (df_files["file_name"].str.contains(chapter_str)) &
                (df_files["file_path"].str.contains(f"{year_str}/"))
            ]

            if match.empty:
                print(f"⚠️ Chapter {chapter_str} not found for year {year_str}")
                continue

            file_id = match.iloc[0]["file_id"]
            file_name = match.iloc[0]["file_name"]
            local_path = os.path.join(download_dir, year_str, f"{chapter_str}_{file_name}")

            # Ensure folder exists
            os.makedirs(os.path.dirname(local_path), exist_ok=True)

            # Download file
            request = drive_service.files().get_media(fileId=file_id)
            fh = io.FileIO(local_path, "wb")
            downloader = MediaIoBaseDownload(fh, request)

            done = False
            while not done:
                status, done = downloader.next_chunk()
                if status:
                    print(f"⬇️ Download {int(status.progress() * 100)}% for {file_name}")

            print(f"✅ Saved {file_name} to {local_path}")
            all_paths[year_str][chapter] = local_path

    return all_paths
download_all_chapters()

⬇️ Download 100% for 01.doc
✅ Saved 01.doc to /content/reports/2001/01_01.doc
⬇️ Download 100% for 02.doc
✅ Saved 02.doc to /content/reports/2001/02_02.doc
⬇️ Download 100% for 03.doc
✅ Saved 03.doc to /content/reports/2001/03_03.doc
⬇️ Download 100% for 04.doc
✅ Saved 04.doc to /content/reports/2001/04_04.doc
⬇️ Download 100% for 05.doc
✅ Saved 05.doc to /content/reports/2001/05_05.doc
⬇️ Download 100% for 06.doc
✅ Saved 06.doc to /content/reports/2001/06_06.doc
⬇️ Download 100% for 07.doc
✅ Saved 07.doc to /content/reports/2001/07_07.doc
⬇️ Download 100% for 08.doc
✅ Saved 08.doc to /content/reports/2001/08_08.doc
⬇️ Download 100% for 09.doc
✅ Saved 09.doc to /content/reports/2001/09_09.doc
⬇️ Download 100% for 10.doc
✅ Saved 10.doc to /content/reports/2001/10_10.doc
⬇️ Download 100% for 11.doc
✅ Saved 11.doc to /content/reports/2001/11_11.doc
⬇️ Download 100% for 12.doc
✅ Saved 12.doc to /content/reports/2001/12_12.doc
⬇️ Download 100% for 13.doc
✅ Saved 13.doc to /content/reports/2

{'2001': {1: '/content/reports/2001/01_01.doc',
  2: '/content/reports/2001/02_02.doc',
  3: '/content/reports/2001/03_03.doc',
  4: '/content/reports/2001/04_04.doc',
  5: '/content/reports/2001/05_05.doc',
  6: '/content/reports/2001/06_06.doc',
  7: '/content/reports/2001/07_07.doc',
  8: '/content/reports/2001/08_08.doc',
  9: '/content/reports/2001/09_09.doc',
  10: '/content/reports/2001/10_10.doc',
  11: '/content/reports/2001/11_11.doc',
  12: '/content/reports/2001/12_12.doc',
  13: '/content/reports/2001/13_13.doc',
  14: '/content/reports/2001/14_14.doc',
  15: '/content/reports/2001/15_15.doc'},
 '2002': {1: '/content/reports/2002/01_01.doc',
  2: '/content/reports/2002/02_02.doc',
  3: '/content/reports/2002/03_03.doc',
  4: '/content/reports/2002/04_04.doc',
  5: '/content/reports/2002/05_05.doc',
  6: '/content/reports/2002/06_06.doc',
  7: '/content/reports/2002/07_07.doc',
  8: '/content/reports/2002/08_08.doc',
  9: '/content/reports/2002/09_09.doc',
  10: '/content/r

In [13]:
import os
import subprocess
from docx import Document

def convert_doc_to_docx(doc_path):
    """
    Convert a .doc file to .docx using LibreOffice (headless mode)
    Returns the path to the converted .docx file
    """
    if not doc_path.endswith(".doc"):
        return doc_path  # already .docx

    converted_path = doc_path + "x"  # simple .docx path
    os.makedirs(os.path.dirname(converted_path), exist_ok=True)

    # LibreOffice command-line conversion
    subprocess.run([
        "libreoffice",
        "--headless",
        "--convert-to",
        "docx",
        "--outdir",
        os.path.dirname(doc_path),
        doc_path
    ], check=True)

    return converted_path

def convert_wmf_to_png(wmf_path, png_path):
    """Convert .wmf or .x-wmf to .png using ImageMagick"""
    subprocess.run([
        "convert", wmf_path, png_path
    ], check=True)

def extract_images_from_docx(docx_path, output_dir):
    """
    Extracts all images from a .docx file and saves them in output_dir.
    WMF images are automatically converted to PNG.
    """
    doc = Document(docx_path)
    image_count = 0
    for i, rel in enumerate(doc.part._rels):
        rel = doc.part._rels[rel]
        if "image" in rel.target_ref:
            image_count += 1
            img_data = rel.target_part.blob
            ext = rel.target_part.content_type.split('/')[-1]
            if ext == 'jpeg':
                ext = 'jpg'

            img_name = f"image_{i+1}.{ext}"
            img_path = os.path.join(output_dir, img_name)
            os.makedirs(output_dir, exist_ok=True)
            with open(img_path, "wb") as f:
                f.write(img_data)

            # Convert WMF to PNG
            if ext in ["x-wmf", "wmf"]:
                png_path = os.path.splitext(img_path)[0] + ".png"
                try:
                    convert_wmf_to_png(img_path, png_path)
                    os.remove(img_path)  # remove original WMF
                    print(f"🔄 Converted {img_path} → {png_path}")
                except Exception as e:
                    print(f"❌ Failed to convert {img_path}: {e}")

    return image_count

# Make sure LibreOffice & ImageMagick are installed
!apt-get update
!apt-get install -y libreoffice imagemagick

# Paths
reports_dir = "/content/reports"          # local folder with reports
images_output_dir = "/content/reports_images"

years = [str(y) for y in range(2001, 2025)]
chapters = list(range(1, 16))  # chapters 1-15

for year in years:
    for chapter in chapters:
        chapter_str = f"{chapter:02}"
        year_folder = os.path.join(reports_dir, year)
        if not os.path.exists(year_folder):
            continue

        # Find all .doc and .docx files for this chapter
        chapter_files = [f for f in os.listdir(year_folder)
                         if f.startswith(chapter_str) and (f.endswith(".docx") or f.endswith(".doc"))]

        if not chapter_files:
            print(f"⚠️ No Word files found for year {year}, chapter {chapter_str}")
            continue

        for file_name in chapter_files:
            file_path = os.path.join(year_folder, file_name)
            # Convert .doc to .docx if needed
            if file_path.endswith(".doc"):
                print(f"🔄 Converting {file_path} to .docx")
                try:
                    file_path = convert_doc_to_docx(file_path)
                except Exception as e:
                    print(f"❌ Conversion failed for {file_path}: {e}")
                    continue

            out_dir = os.path.join(images_output_dir, year, chapter_str)
            count = extract_images_from_docx(file_path, out_dir)
            print(f"🖼 Extracted {count} images from {file_path} to {out_dir}")


Streaming output truncated to the last 5000 lines.
🔄 Converted /content/reports_images/2001/02/image_3.x-wmf → /content/reports_images/2001/02/image_3.png
🔄 Converted /content/reports_images/2001/02/image_10.x-wmf → /content/reports_images/2001/02/image_10.png
🔄 Converted /content/reports_images/2001/02/image_11.x-wmf → /content/reports_images/2001/02/image_11.png
🔄 Converted /content/reports_images/2001/02/image_12.x-wmf → /content/reports_images/2001/02/image_12.png
🔄 Converted /content/reports_images/2001/02/image_13.x-wmf → /content/reports_images/2001/02/image_13.png
🔄 Converted /content/reports_images/2001/02/image_14.x-wmf → /content/reports_images/2001/02/image_14.png
🔄 Converted /content/reports_images/2001/02/image_15.x-wmf → /content/reports_images/2001/02/image_15.png
🖼 Extracted 7 images from /content/reports/2001/02_02.docx to /content/reports_images/2001/02
🔄 Converted /content/reports_images/2001/02/image_3.x-wmf → /content/reports_images/2001/02/image_3.png
🔄 Converted

In [15]:
import os
import subprocess
import zipfile
from collections import Counter

def convert_doc_to_docx(doc_path):
    """Convert a .doc file to .docx using LibreOffice headless."""
    if not doc_path.endswith(".doc"):
        return doc_path

    converted_path = doc_path + "x"
    subprocess.run([
        "libreoffice", "--headless", "--convert-to", "docx",
        "--outdir", os.path.dirname(doc_path), doc_path
    ], check=True)

    return converted_path

def analyze_docx_content(docx_path):
    """
    Analyze a .docx file to count charts vs images.
    Returns a dict: {"charts": X, "images": Y}
    """
    counts = Counter({"charts": 0, "images": 0})
    with zipfile.ZipFile(docx_path, "r") as docx_zip:
        for f in docx_zip.namelist():
            if f.startswith("word/charts/") and f.endswith(".xml"):
                counts["charts"] += 1
            elif f.startswith("word/media/"):
                counts["images"] += 1
    return counts

# Example usage
reports_dir = "/content/reports"
years = [str(y) for y in range(2001, 2025)]
chapters = [1, 2]

for year in years:
    for chapter in chapters:
        chapter_str = f"{chapter:02}"
        year_folder = os.path.join(reports_dir, year)
        if not os.path.exists(year_folder):
            continue

        chapter_files = [f for f in os.listdir(year_folder)
                         if f.startswith(chapter_str) and (f.endswith(".docx") or f.endswith(".doc"))]

        for file_name in chapter_files:
            file_path = os.path.join(year_folder, file_name)

            if file_path.endswith(".doc"):
                print(f"🔄 Converting {file_path} to .docx")
                try:
                    file_path = convert_doc_to_docx(file_path)
                except Exception as e:
                    print(f"❌ Conversion failed for {file_path}: {e}")
                    continue

            stats = analyze_docx_content(file_path)
            print(f"📊 {file_name} → Charts: {stats['charts']}, Images: {stats['images']}")


📊 01_01.docx → Charts: 0, Images: 3
🔄 Converting /content/reports/2001/01_01.doc to .docx
📊 01_01.doc → Charts: 0, Images: 3
🔄 Converting /content/reports/2001/02_02.doc to .docx
📊 02_02.doc → Charts: 0, Images: 7
📊 02_02.docx → Charts: 0, Images: 7
📊 01_01.docx → Charts: 0, Images: 6
🔄 Converting /content/reports/2002/01_01.doc to .docx
📊 01_01.doc → Charts: 0, Images: 6
🔄 Converting /content/reports/2002/02_02.doc to .docx
📊 02_02.doc → Charts: 0, Images: 7
📊 02_02.docx → Charts: 0, Images: 7
📊 01_01.docx → Charts: 0, Images: 6
🔄 Converting /content/reports/2003/01_01.doc to .docx
📊 01_01.doc → Charts: 0, Images: 6
🔄 Converting /content/reports/2003/02_02.doc to .docx
📊 02_02.doc → Charts: 0, Images: 7
📊 02_02.docx → Charts: 0, Images: 7
📊 01_01.docx → Charts: 0, Images: 6
🔄 Converting /content/reports/2004/01_01.doc to .docx
📊 01_01.doc → Charts: 0, Images: 6
🔄 Converting /content/reports/2004/02_02.doc to .docx
📊 02_02.doc → Charts: 0, Images: 9
📊 02_02.docx → Charts: 0, Images: 9
